In [ ]:
import os
import numpy as np
import pandas as pd
import random


from transformers import (AdamW, get_linear_schedule_with_warmup, logging, 
                          BigBirdConfig, BigBirdTokenizer, BigBirdForSequenceClassification, 
                          BigBirdPreTrainedModel, BigBirdModel)
from transformers.activations import ACT2FN

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, SequentialSampler, RandomSampler, DataLoader

from tqdm.notebook import tqdm

import gc; gc.enable()
from IPython.display import clear_output

from sklearn.model_selection import StratifiedKFold

logging.set_verbosity_error()

In [ ]:
INPUT_DIR = '../input/commonlitreadabilityprize'
MODEL_NAME = 'roberta-large'

MAX_LENGTH = 256
LR = 2e-5
EPS = 1e-8

SEED = 42

NUM_FOLDS = 5
SEEDS = [113, 71, 17, 43, 37]

EPOCHS = 5
TRAIN_BATCH_SIZE = 8
VAL_BATCH_SIZE = 32

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def set_seed(seed = 0):
    np.random.seed(seed)
    random_state = np.random.RandomState(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    return random_state

random_state = set_seed(SEED)

In [ ]:
class ContinuousStratifiedKFold(StratifiedKFold):
    def split(self, x, y, groups=None):
        num_bins = int(np.floor(1 + np.log2(len(y))))
        bins = pd.cut(y, bins=num_bins, labels=False)
        return super().split(x, bins, groups)
    
def get_data_loaders(data, fold):
    
    x_train = data.loc[data.fold != fold, 'excerpt'].tolist()
    y_train = data.loc[data.fold != fold, 'target'].values
    x_val = data.loc[data.fold == fold, 'excerpt'].tolist()
    y_val = data.loc[data.fold == fold, 'target'].values
    
    encoded_train = tokenizer.batch_encode_plus(
        x_train, 
        add_special_tokens=True, 
        return_attention_mask=True, 
        padding='max_length', 
        truncation=True,
        max_length=MAX_LENGTH, 
        return_tensors='pt'
    )
    
    encoded_val = tokenizer.batch_encode_plus(
        x_val, 
        add_special_tokens=True, 
        return_attention_mask=True, 
        padding='max_length', 
        truncation=True,
        max_length=MAX_LENGTH, 
        return_tensors='pt'
    )
    
    dataset_train = TensorDataset(
        encoded_train['input_ids'],
        encoded_train['attention_mask'],
        torch.tensor(y_train)
    )
    dataset_val = TensorDataset(
        encoded_val['input_ids'],
        encoded_val['attention_mask'],
        torch.tensor(y_val)
    )
    
    dataloader_train = DataLoader(
        dataset_train,
        sampler = RandomSampler(dataset_train),
        batch_size=TRAIN_BATCH_SIZE
    )

    dataloader_val = DataLoader(
        dataset_val,
        sampler = SequentialSampler(dataset_val),
        batch_size=VAL_BATCH_SIZE
    )

    return dataloader_train, dataloader_val

In [ ]:
data = pd.read_csv(os.path.join(INPUT_DIR, 'train.csv'))

# Create stratified folds
kf = ContinuousStratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
for f, (t_, v_) in enumerate(kf.split(data, data.target)):
    data.loc[v_, 'fold'] = f
data['fold'] = data['fold'].astype(int)

In [ ]:
def evaluate(model, val_dataloader):
    model.eval()
    loss_val_total = 0
    for batch in val_dataloader:
        batch = tuple(b.to(DEVICE) for b in batch)
        labels = batch[2].type(torch.float)
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'labels': labels,
                 }
        with torch.no_grad():
            # To be used when using default architecture from transformers
#             output = model(**inputs)
#             loss = output.loss
            # To be used with custome head
            loss = model(**inputs)
        loss_val_total += loss.item()
    loss_val_avg = loss_val_total/len(val_dataloader) 
    return loss_val_avg

def train(model, train_dataloader, val_dataloader):
    optimizer = AdamW(model.parameters(), lr = LR, eps = EPS)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * EPOCHS)
    best_val_loss = 1
    model.train()
    for epoch in range(EPOCHS):
        loss_train_total = 0
        for batch in tqdm(train_dataloader):
            model.zero_grad()
            batch = tuple(b.to(DEVICE) for b in batch)
            labels = batch[2].type(torch.float)
            inputs = {
                'input_ids': batch[0],
                'attention_mask': batch[1],
                'labels': labels,
            }
            # To be used when using default architecture from transformers
#             output = model(**inputs)
#             loss = output.loss
            # To be used with custome head
            loss = model(**inputs)
            loss_train_total += loss.item()
            loss.backward()
            optimizer.step()
            scheduler.step()
        loss_train_avg = loss_train_total / len(train_dataloader)
        loss_val_avg = evaluate(model, val_dataloader)
        print(f'epoch:{epoch+1}/{EPOCHS} train loss={loss_train_avg}  val loss={loss_val_avg}')
        
        if loss_val_avg < best_val_loss:
            best_val_loss = loss_val_avg    
    return best_val_loss

In [ ]:
class BigBirdForRegression(BigBirdPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config
        self.bert = BigBirdModel(config)
        self.dropout = nn.Dropout(0.1)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)
        self.loss = nn.MSELoss()

        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        last_hidden_state = outputs[0]
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        
        dropped = self.dropout(mean_embeddings)
        logits = self.out_proj(dropped)

        preds = logits.squeeze(-1).squeeze(-1)
        
        if labels is not None:
            loss = self.loss(preds.view(-1).float(), labels.view(-1).float())
            return loss
        else:
            return preds

In [ ]:
tokenizer = BigBirdTokenizer.from_pretrained('google/bigbird-roberta-large')
config = BigBirdConfig.from_pretrained('google/bigbird-roberta-large')
config.update({'problem_type': 'regression', 'num_labels': 1})
# model = BigBirdForSequenceClassification.from_pretrained("google/bigbird-roberta-large", config=config)
model = BigBirdForRegression.from_pretrained("google/bigbird-roberta-large", config=config)
model.to(DEVICE)
clear_output()

In [ ]:
losses = []

MAX_RUNS = 2
runs = 0   # Variable to control termination condition

for i, seed in enumerate(SEEDS):       
    # Termination condition
    if runs == MAX_RUNS:
        print(f'{runs} runs termination condition reached.')
        break    
    
    print(f'********* seed({i}) = {seed} ***********')
    
    for fold in range(NUM_FOLDS):
        print(f'*** fold = {fold} ***')
        set_seed(seed)
        train_dataloader, val_dataloader = get_data_loaders(data, fold)
            
        loss = train(model, train_dataloader, val_dataloader)
        losses.append(loss)
        
        # Termination condition
        runs += 1
        if runs == MAX_RUNS:
            break

In [ ]:
from sklearn.metrics import mean_squared_error

train_dataloader, val_dataloader = get_data_loaders(data, 2)
model.eval()
predictions = []
targets = []
with torch.no_grad():
    for batch in val_dataloader:
        batch = tuple(b.to(DEVICE) for b in batch)
        targets.extend(batch[2].cpu().detach().numpy().ravel().tolist())
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
        }
        outputs = model(**inputs)
        predictions.extend(outputs.cpu().detach().numpy().ravel().tolist()) #Custome head
#         predictions.extend(outputs.logits.cpu().detach().numpy().ravel().tolist()) #Default head

mean_squared_error(targets, predictions, squared=False)
# Default head score
# 0.08338136126320507
# Custome head score
# 0.09466690151432648